In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

## Header
Wo wir unseren geheimen Schlüssel für ChatGPT mitschicken mit dem wir uns automatisch einloggen und authentifizieren (und auch unser Budget ansprechen)

In [ ]:
import os
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ["API_KEY"]}",
}

Die Daten die wir mit unsere Anfrage schicken - dies entspricht der GESAMTEN Unterhaltung!

In [ ]:
payload = {
  # Die -mini Modelle sind günstiger per token. Fürs explorieren, macht es Sinn ein günstiges Modell zu benutzen.
  "model": "gpt-4o-mini",
  "messages": [
    {
        "role": "system", 
        "content": "Du bist ein Poet. Du formulierst mir ein Gedicht basierend auf den Nutzereingaben."
    },
    {
      "role": "user",
      # Achtung: Alle informationen die wir hier an OpenAI schicken, werden gespeichert von OpenAI. 
      # Keine sensiblen Informationen oder Details über die eigene/dritte Personen bzw. Informationen mitschicken, 
      # wo man nicht die Rechte dran besitzt.
      "content": "Das Abendbrot war lecker."
    }
  ],
  "temperature": 0.5
}

In [ ]:
import pprint
import requests
ENDPOINT = "https://api.openai.com/v1/chat/completions"
    
response = requests.post(ENDPOINT, headers=headers, json=payload)
if response.status_code != 200:
    logger.warning(f"Fehlerhafte Antwort: {response.text}")

### Rohausgabe der Rückantwort

In [ ]:
endpoint_response = response.json()
pprint.pp(endpoint_response)

Extrahieren der eigentlichen Antwort aus der Rückantwort des Endpunkt

In [ ]:
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer